In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# orig_url = "https://drive.google.com/file/d/12fFZ9k8wsmWBVUhcsVxmKsqHxaVzAzqt/view?usp=sharing"
# file_id = orig_url.split('/')[-2]
# data_path='https://drive.google.com/uc?export=download&id=' + file_id

data_path='https://drive.google.com/uc?export=download&id=1A3MUldrs0z09DlYR6Y1utfySwKNO9Qsz'
df = pd.read_csv(data_path)
df.head(10)

,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,4528,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No
2,6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.5,No
4,432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.5,No
5,2215,4312-GVYNH,Female,0,Yes,No,70,No,No phone service,DSL,...,Yes,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),49.85,3370.2,No
6,5260,2495-KZNFB,Female,0,No,No,33,Yes,Yes,Fiber optic,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,90.65,2989.6,No
7,6001,4367-NHWMM,Female,0,No,No,1,No,No phone service,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,24.90,24.9,No
8,1480,8898-KASCD,Male,0,No,No,39,No,No phone service,DSL,...,Yes,Yes,No,No,One year,No,Mailed check,35.55,1309.15,No
9,5137,8016-NCFVO,Male,1,No,No,55,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,116.50,6382.55,No


In [ ]:
df["SeniorCitizen"] = df["SeniorCitizen"].astype("object")
df["TotalCharges"]= df["TotalCharges"].apply(lambda x: "0" if x == " " else x )
df["TotalCharges"] = df["TotalCharges"].astype("float64")

In [ ]:
df["num_services"] = (
    df["PhoneService"].apply(lambda x: 1 if x == "Yes" else 0 )+
    df["MultipleLines"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["InternetService"].apply(lambda x: 0 if x == "No" else 1 ) +
    df["OnlineSecurity"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["OnlineBackup"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["DeviceProtection"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["TechSupport"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["StreamingTV"].apply(lambda x: 1 if x == "Yes" else 0 ) +
    df["StreamingMovies"].apply(lambda x: 1 if x == "Yes" else 0 )
)
df["num_services"]

0       2
1       5
2       3
3       3
4       4
       ..
5981    5
5982    9
5983    1
5984    6
5985    1
Name: num_services, Length: 5986, dtype: int64

In [ ]:
df["auto_payment"] = df["PaymentMethod"].apply(lambda x: 1 if "automatic" in x  else 0 )
df["auto_payment"] = df["auto_payment"].astype("object")

In [ ]:
X = df.drop(columns = ['Unnamed: 0','Churn', "customerID", 'auto_payment', 'num_services'])
y = df['Churn']
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4190, 19), (1796, 19), (4190,), (1796,))

In [ ]:
X_num_services = df.drop(columns=['Unnamed: 0','Churn', "customerID", 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies'])
X_num_services.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5986 entries, 0 to 5985
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5986 non-null   object 
 1   SeniorCitizen     5986 non-null   object 
 2   Partner           5986 non-null   object 
 3   Dependents        5986 non-null   object 
 4   tenure            5986 non-null   int64  
 5   Contract          5986 non-null   object 
 6   PaperlessBilling  5986 non-null   object 
 7   PaymentMethod     5986 non-null   object 
 8   MonthlyCharges    5986 non-null   float64
 9   TotalCharges      5986 non-null   float64
 10  num_services      5986 non-null   int64  
 11  auto_payment      5986 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 561.3+ KB


In [ ]:
# Removed Payment method
X_auto = df.drop(columns=['Unnamed: 0','Churn', "customerID"])
X_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5986 entries, 0 to 5985
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5986 non-null   object 
 1   SeniorCitizen     5986 non-null   object 
 2   Partner           5986 non-null   object 
 3   Dependents        5986 non-null   object 
 4   tenure            5986 non-null   int64  
 5   PhoneService      5986 non-null   object 
 6   MultipleLines     5986 non-null   object 
 7   InternetService   5986 non-null   object 
 8   OnlineSecurity    5986 non-null   object 
 9   OnlineBackup      5986 non-null   object 
 10  DeviceProtection  5986 non-null   object 
 11  TechSupport       5986 non-null   object 
 12  StreamingTV       5986 non-null   object 
 13  StreamingMovies   5986 non-null   object 
 14  Contract          5986 non-null   object 
 15  PaperlessBilling  5986 non-null   object 
 16  PaymentMethod     5986 non-null   object 


In [ ]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.3-py2.py3-none-any.whl size=265293968 sha256=70a6c2f0093fbc15a18eb0cc0b30cc87856b575a53f518f6341ea17e712f79ac
  Stored in directory: /root/.cache/pip/wheels/77/9a/1c/2da26f943fd46b57f3c20b54847b936b9152b831dc7447cf71
Successfully built h2o


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmin9752k
  JVM stdout: /tmp/tmpmin9752k/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpmin9752k/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,08 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_unknownUser_jyxls9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(X_num_services, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4190, 12), (1796, 12), (4190,), (1796,))

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1)
train_data = h2o.H2OFrame.from_python(train_data)
train_data['Churn'] = train_data['Churn'].asfactor()
train_data.head()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


gender,SeniorCitizen,Partner,Dependents,tenure,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,num_services,auto_payment,Churn
Male,0,No,No,27,One year,No,Bank transfer (automatic),20.1,562.6,1,1,No
Male,0,No,No,63,One year,No,Credit card (automatic),104.5,6590.8,8,1,No
Male,0,Yes,No,71,Two year,Yes,Credit card (automatic),108.6,7690.9,8,1,Yes
Female,0,Yes,Yes,12,One year,No,Credit card (automatic),33.15,444.75,3,1,No
Male,0,Yes,No,14,Month-to-month,Yes,Electronic check,86.1,1235.55,4,0,Yes
Female,1,Yes,No,62,One year,Yes,Credit card (automatic),97.95,5936.55,6,1,No
Male,0,No,No,30,Month-to-month,Yes,Credit card (automatic),94.3,2679.7,6,1,No
Female,0,No,No,59,Month-to-month,Yes,Bank transfer (automatic),99.45,5623.7,7,1,No
Male,1,Yes,No,30,Month-to-month,Yes,Electronic check,101.5,2917.65,6,0,No
Male,0,Yes,No,48,Two year,Yes,Bank transfer (automatic),96.9,4473.45,7,1,No


In [ ]:
aml = H2OAutoML(max_models=30, # hard total models experimented limit
                max_runtime_secs=300, # try to run as many models in 5 mins
                max_runtime_secs_per_model=30, # max training time for a model <= 30 s else move to next model
                seed=1)
aml.train(x=X_train.columns.tolist(), y=y_train.name, training_frame=train_data)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),11/30
# GBM base models (used / total),5/10
# XGBoost base models (used / total),2/10
# GLM base models (used / total),1/1
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),3/7
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [ ]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb.head(lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_7_20231222_104634,0.841225,0.420496,0.64528,0.236986,0.369838,0.13678,10260,0.11254,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_7_20231222_104634,0.84093,0.42129,0.641167,0.238577,0.369953,0.136866,6753,0.041257,StackedEnsemble
GBM_grid_1_AutoML_7_20231222_104634_model_2,0.840343,0.421819,0.645723,0.232741,0.370146,0.137008,560,0.011814,GBM
GBM_1_AutoML_7_20231222_104634,0.840166,0.421856,0.640585,0.239408,0.370513,0.13728,441,0.015804,GBM
GBM_grid_1_AutoML_7_20231222_104634_model_3,0.837171,0.425198,0.643396,0.241741,0.37204,0.138414,272,0.014245,GBM
GBM_grid_1_AutoML_7_20231222_104634_model_4,0.836091,0.426633,0.636929,0.247972,0.372166,0.138508,410,0.017786,GBM
GBM_5_AutoML_7_20231222_104634,0.835636,0.427199,0.638088,0.244562,0.373113,0.139214,313,0.013214,GBM
GLM_1_AutoML_7_20231222_104634,0.834257,0.428799,0.623717,0.2489,0.37409,0.139944,155,0.003169,GLM
XGBoost_grid_1_AutoML_7_20231222_104634_model_6,0.832759,0.434414,0.626145,0.245286,0.375792,0.141219,912,0.006956,XGBoost
GBM_2_AutoML_7_20231222_104634,0.830376,0.433745,0.618601,0.246794,0.376104,0.141454,381,0.013097,GBM


In [ ]:
# leader_model = aml.get_best_model(algorithm="GBM")
leader_model = aml.leader
leader_model.get_params()

{'model_id': None,
 'training_frame': None,
 'response_column': None,
 'validation_frame': None,
 'blending_frame': None,
 'base_models': [],
 'metalearner_algorithm': 'auto',
 'metalearner_nfolds': 0,
 'metalearner_fold_assignment': None,
 'metalearner_fold_column': None,
 'metalearner_params': None,
 'metalearner_transform': 'none',
 'max_runtime_secs': 0.0,
 'weights_column': None,
 'offset_column': None,
 'custom_metric_func': None,
 'seed': -1,
 'score_training_samples': 10000,
 'keep_levelone_frame': False,
 'export_checkpoints_dir': None,
 'auc_type': 'auto',
 'gainslift_bins': -1}

In [ ]:
X_test_h2o = h2o.H2OFrame.from_python(X_test)
y_pred = leader_model.predict(X_test_h2o)
y_pred = y_pred.as_data_frame()['predict'].values

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [ ]:
print(classification_report(y_test, y_pred), confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

          No       0.90      0.74      0.81      1320
         Yes       0.52      0.78      0.63       476

    accuracy                           0.75      1796
   macro avg       0.71      0.76      0.72      1796
weighted avg       0.80      0.75      0.76      1796
 [[978 342]
 [104 372]]


AUTO ML again with X_auto

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(X_auto, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4190, 20), (1796, 20), (4190,), (1796,))

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1)
train_data = h2o.H2OFrame.from_python(train_data)
train_data['Churn'] = train_data['Churn'].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
aml = H2OAutoML(max_models=30, # hard total models experimented limit
                max_runtime_secs=300, # try to run as many models in 5 mins
                max_runtime_secs_per_model=30, # max training time for a model <= 30 s else move to next model
                seed=1)
aml.train(x=X_train.columns.tolist(), y=y_train.name, training_frame=train_data)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_4_20231222_100547_model_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    38                 38                          9741                   4            4            4             14            16            15.8158

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.12027528196115064
RMSE: 0.34680726918729754
LogLoss: 0.376792448214931
Mean Per-Class Error: 0.1968802921214499
AUC: 0.8822732549318588
AUCPR: 0.7304771260995686
Gini: 0.7645465098637176

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3122019771730514
       No    Yes    Error    Rate
-----  ----  -----  -------  --------------
No     2468  611    0.1984   (611.0/3079.0)
Yes    217   894    0.1953   (217.0/1111.0)
Total  2685  1505   0.1976   (828.0/4190.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.312202     0.683486  216
max f2                       0.182716     0.777337  285
max f0point5                 0.519644     0.691773  127
max accuracy                 0.499633     0.830072  136
max precision                0.894689     1         0
max recall                   0.0253527    1         394
max specificity              0.894689     1         0
max absolute_mcc             0.312202     0.557783  216
max min_per_class_accuracy   0.312202     0.801559  216
max mean_per_class_accuracy  0.30429      0.804099  221
max tns                      0.894689     3079      0
max fns                      0.894689     1110      0
max fps                      0.0202681    3079      399
max tps                      0.0253527    1111      394
max tnr                      0.894689     1         0
max fnr                      0.894689     0.9991    0
max fpr                      0.0202681    1         399
max tpr                      0.0253527    1         394

Gains/Lift Table: Avg response rate: 26.52 %, avg score: 26.54 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100239                   0.829522           3.68158     3.68158            0.97619          0.854277   0.97619                     0.854277            0.0369037       0.0369037                  268.158   268.158            0.0365789
2        0.0202864                   0.80625            3.33284     3.50516            0.883721         0.814507   0.929412                    0.834158            0.0342034       0.0711071                  233.284   250.516            0.0691584
3        0.0300716                   0.787409           3.31145     3.44213            0.878049         0.795501   0.912698                    0.821579            0.0324032       0.10351                    231.145   244.213            0.0999378
4        0.0400955                   0.769126           3.50199     3.4571             0.928571         0.779679   0.916667                    0.811104            0.0351035       0.138614                   250.199   245.71             0.134067
5        0.0501193                   0.749677           3.14281     3.39424            0.833333         0.759064   0.

In [ ]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb.head(lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
GBM_grid_1_AutoML_4_20231222_100547_model_2,0.84437,0.416947,0.659684,0.237645,0.367684,0.135192,347,0.012929,GBM
GBM_1_AutoML_4_20231222_100547,0.844137,0.416574,0.661877,0.246691,0.367309,0.134916,460,0.014303,GBM
GLM_1_AutoML_4_20231222_100547,0.840951,0.42178,0.646835,0.235807,0.369998,0.136899,232,0.003095,GLM
GBM_grid_1_AutoML_4_20231222_100547_model_4,0.840471,0.420893,0.657575,0.235473,0.368658,0.135909,464,0.015732,GBM
GBM_5_AutoML_4_20231222_100547,0.840283,0.421552,0.652155,0.241184,0.370091,0.136967,582,0.012978,GBM
GBM_grid_1_AutoML_4_20231222_100547_model_3,0.839238,0.42171,0.659378,0.245703,0.369439,0.136485,324,0.014608,GBM
XGBoost_grid_1_AutoML_4_20231222_100547_model_7,0.836114,0.428222,0.650837,0.252004,0.372132,0.138482,692,0.007123,XGBoost
GBM_2_AutoML_4_20231222_100547,0.83521,0.427377,0.642497,0.245736,0.372443,0.138714,940,0.014914,GBM
XGBoost_grid_1_AutoML_4_20231222_100547_model_2,0.833357,0.43502,0.63916,0.243801,0.37548,0.140986,819,0.008944,XGBoost
GBM_grid_1_AutoML_4_20231222_100547_model_1,0.833153,0.429311,0.639074,0.242038,0.372886,0.139044,610,0.017173,GBM


In [ ]:
leader_model = aml.leader
leader_model.get_params()

{'model_id': None,
 'training_frame': None,
 'validation_frame': None,
 'nfolds': 0,
 'keep_cross_validation_models': True,
 'keep_cross_validation_predictions': False,
 'keep_cross_validation_fold_assignment': False,
 'score_each_iteration': False,
 'score_tree_interval': 0,
 'fold_assignment': 'auto',
 'fold_column': None,
 'response_column': None,
 'ignored_columns': None,
 'ignore_const_cols': True,
 'offset_column': None,
 'weights_column': None,
 'balance_classes': False,
 'class_sampling_factors': None,
 'max_after_balance_size': 5.0,
 'max_confusion_matrix_size': 20,
 'ntrees': 50,
 'max_depth': 5,
 'min_rows': 10.0,
 'nbins': 20,
 'nbins_top_level': 1024,
 'nbins_cats': 1024,
 'r2_stopping': None,
 'stopping_rounds': 0,
 'stopping_metric': 'auto',
 'stopping_tolerance': 0.001,
 'max_runtime_secs': 0.0,
 'seed': -1,
 'build_tree_one_node': False,
 'learn_rate': 0.1,
 'learn_rate_annealing': 1.0,
 'distribution': 'auto',
 'quantile_alpha': 0.5,
 'tweedie_power': 1.5,
 'huber_alp

In [ ]:
X_test_h2o = h2o.H2OFrame.from_python(X_test)
y_pred = leader_model.predict(X_test_h2o)
y_pred = y_pred.as_data_frame()['predict'].values

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [ ]:
print(classification_report(y_test, y_pred), confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

          No       0.89      0.78      0.83      1320
         Yes       0.55      0.74      0.63       476

    accuracy                           0.77      1796
   macro avg       0.72      0.76      0.73      1796
weighted avg       0.80      0.77      0.78      1796
 [[1031  289]
 [ 123  353]]


## We do our own GBM model

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(X_auto, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4190, 20), (1796, 20), (4190,), (1796,))

In [ ]:
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_features = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
# categorical_features.append("tenure")
# numeric_features.remove("tenure")
categorical_features, numeric_features

(['gender',
  'SeniorCitizen',
  'Partner',
  'Dependents',
  'PhoneService',
  'MultipleLines',
  'InternetService',
  'OnlineSecurity',
  'OnlineBackup',
  'DeviceProtection',
  'TechSupport',
  'StreamingTV',
  'StreamingMovies',
  'Contract',
  'PaperlessBilling',
  'auto_payment'],
 ['tenure', 'MonthlyCharges', 'TotalCharges', 'num_services'])

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
categorical_transformer = Pipeline(steps=[
                                          ("cat_imputer", SimpleImputer(strategy='constant',
                                                                        fill_value='Not Available').set_output(transform="pandas")),
                                          ("onehot", OneHotEncoder(sparse_output=False,
                                                                   handle_unknown="ignore", drop = "if_binary" ).set_output(transform="pandas"))
                                          ])
numeric_transformer = Pipeline(steps=[
                                      ("knn_imputer", KNNImputer(n_neighbors=5).set_output(transform="pandas")),
                                      ("scaler", MinMaxScaler().set_output(transform="pandas"))
                                      ])
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
                                               ("num", numeric_transformer,
                                                       numeric_features),
                                               ("cat", categorical_transformer,
                                                       categorical_features)
                                               ]).set_output(transform="pandas")
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('knn_imputer', KNNImputer()),
                                                 ('scaler', MinMaxScaler())]),
                                 ['tenure', 'MonthlyCharges', 'TotalCharges',
                                  'num_services']),
                                ('cat',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(fill_value='Not '
                                                                           'Available',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['gender', 'SeniorCitizen', 'Partner',
                                  'Dependents', 'PhoneService', 'MultipleLines',
                                  'InternetService', 'OnlineSecurity',
                                  'OnlineBackup', 'DeviceProtection',
                                  'TechSupport', 'StreamingTV',
                                  'StreamingMovies', 'Contract',
                                  'PaperlessBilling', 'auto_payment'])])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTENC

categorical_indices = [X_auto.columns.get_loc(col) for col in categorical_features]
gbm = GradientBoostingClassifier(random_state=42, max_depth = 5, n_estimators = 50, min_samples_split = 10, subsample = 1, loss = "deviance", max_features = None,min_samples_leaf = 1  )
pipeline_gbm = imbpipeline([("pre_process", preprocessor),('oversampling', SMOTENC(sampling_strategy='auto', categorical_features=categorical_indices, random_state=42)),
                         ("model", gbm)])
pipeline_gbm

Pipeline(steps=[('pre_process',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['tenure', 'MonthlyCharges',
                                                   'TotalCharges',
                                                   'num_services']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'Available',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handl...
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'auto_payment'])])),
                ('oversampling',
                 SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 8, 9, 10,
                                               11, 12, 13, 14, 15, 19],
                         random_state=42)),
                ('model',
                 GradientBoostingClassifier(loss='deviance', max_depth=5,
                                            min_samples_split=10,
                                            n_estimators=50, random_state=42,
                                            subsample=1))])

In [ ]:
# print(np.mean(cross_val_score(pipeline_gbm, X_train, y_train, cv=3, scoring='roc_auc')))
y_train2 = y_train.map({'No': 0, 'Yes': 1})
pipeline_gbm.fit(X_train, y_train)
# y_pred = (pipeline_gbm.predict_proba(X_test)[:,1] >= 0.4)

# y_test2 = y_test.map({'No': 0, 'Yes': 1})
y_pred = pipeline_gbm.predict(X_test)
print(classification_report(y_test, y_pred), confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


              precision    recall  f1-score   support

          No       0.86      0.83      0.85      1320
         Yes       0.58      0.64      0.61       476

    accuracy                           0.78      1796
   macro avg       0.72      0.74      0.73      1796
weighted avg       0.79      0.78      0.78      1796
 [[1099  221]
 [ 172  304]]


In [ ]:
X_list = [X, X_num_services, X_auto]

for Xs in X_list:
  X_train, X_test, y_train, y_test =  train_test_split(Xs, y, test_size=0.3, random_state=42, stratify=y)
  X_train.shape, X_test.shape, y_train.shape, y_test.shape
  categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
  numeric_features = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
  # categorical_features.append("tenure")
  # numeric_features.remove("tenure")
  from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
  from sklearn.model_selection import cross_val_score
  from sklearn.pipeline import Pipeline
  from sklearn.impute import SimpleImputer, KNNImputer
  categorical_transformer = Pipeline(steps=[
                                            ("cat_imputer", SimpleImputer(strategy='constant',
                                                                          fill_value='Not Available').set_output(transform="pandas")),
                                            ("onehot", OneHotEncoder(sparse_output=False,
                                                                    handle_unknown="ignore", drop = "if_binary" ).set_output(transform="pandas"))
                                            ])
  numeric_transformer = Pipeline(steps=[
                                        ("knn_imputer", KNNImputer(n_neighbors=5).set_output(transform="pandas")),
                                        ("scaler", MinMaxScaler().set_output(transform="pandas"))
                                        ])
  from sklearn.compose import ColumnTransformer
  preprocessor = ColumnTransformer(transformers=[
                                                ("num", numeric_transformer,
                                                        numeric_features),
                                                ("cat", categorical_transformer,
                                                        categorical_features)
                                                ]).set_output(transform="pandas")
  from sklearn.ensemble import GradientBoostingClassifier
  from imblearn.over_sampling import SMOTE
  from imblearn.pipeline import Pipeline as imbpipeline
  from imblearn.over_sampling import SMOTENC

  categorical_indices = [Xs.columns.get_loc(col) for col in categorical_features]
  gbm = GradientBoostingClassifier(random_state=42, max_depth = 5, n_estimators = 50, min_samples_split = 10, subsample = 1, loss = "deviance", max_features = None,min_samples_leaf = 1  )
  pipeline_gbm = imbpipeline([("pre_process", preprocessor),('oversampling', SMOTENC(sampling_strategy='auto', categorical_features=categorical_indices, random_state=42)),
                          ("model", gbm)])




  y_train2 = y_train.map({'No': 0, 'Yes': 1})
  pipeline_gbm.fit(X_train, y_train)
  # y_pred = (pipeline_gbm.predict_proba(X_test)[:,1] >= 0.4)

  # y_test2 = y_test.map({'No': 0, 'Yes': 1})
  y_pred = pipeline_gbm.predict(X_test)
  print(classification_report(y_test, y_pred), confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


              precision    recall  f1-score   support

          No       0.87      0.83      0.85      1320
         Yes       0.58      0.65      0.62       476

    accuracy                           0.78      1796
   macro avg       0.73      0.74      0.73      1796
weighted avg       0.79      0.78      0.79      1796
 [[1098  222]
 [ 166  310]]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


              precision    recall  f1-score   support

          No       0.86      0.81      0.83      1320
         Yes       0.54      0.63      0.59       476

    accuracy                           0.76      1796
   macro avg       0.70      0.72      0.71      1796
weighted avg       0.78      0.76      0.77      1796
 [[1066  254]
 [ 174  302]]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


              precision    recall  f1-score   support

          No       0.86      0.83      0.85      1320
         Yes       0.58      0.64      0.61       476

    accuracy                           0.78      1796
   macro avg       0.72      0.74      0.73      1796
weighted avg       0.79      0.78      0.78      1796
 [[1098  222]
 [ 172  304]]
